In [2]:
!pip install transformers

     ---------------------------------------- 7.5/7.5 MB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 2.8 MB/s eta 0:00:00
     -------------------------------------- 268.8/268.8 kB 8.3 MB/s eta 0:00:00
     ------------------------------------- 266.4/266.4 kB 16.0 MB/s eta 0:00:00
     ------------------------------------- 268.4/268.4 kB 16.1 MB/s eta 0:00:00


In [1]:
!pip list

Package                      Version


---------------------------- --------------------
absl-py                      1.4.0
aiohttp                      3.8.3
aiosignal                    1.3.1
albumentations               1.3.0
appdirs                      1.4.4
asttokens                    2.2.1
astunparse                   1.6.3
async-timeout                4.0.2
asyncio                      3.4.3
attrs                        22.2.0
audioread                    3.0.0
backcall                     0.2.0
beautifulsoup4               4.12.0
cachetools                   5.3.0
certifi                      2022.12.7
cffi                         1.15.1
charset-normalizer           2.1.1
click                        8.1.3
cloudpickle                  2.2.1
colorama                     0.4.6
comm                         0.1.2
contourpy                    1.0.7
cycler                       0.11.0
Cython                       0.29.33
dask                         2023.4.0
dask-glm                     0.2.0
dask-ml                   

In [3]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import pandas as pd
import tensorflow as tf

In [4]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [12]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

def convert_data_to_examples(train, DATA_COLUMN, LABEL_COLUMN): 
    return train.apply(lambda x: InputExample(guid=None, text_a = x[DATA_COLUMN], text_b = None, label = x[LABEL_COLUMN]), axis = 1)

def convert_data_to_examples_without_labels(test, DATA_COLUMN): 
    return test.apply(lambda x: InputExample(guid=None, text_a = x[DATA_COLUMN], text_b = None, label = 0), axis = 1)

def convert_examples_to_tf_dataset(examples, tokenizer):
    features = []
    for e in examples:
        input_dict = tokenizer.encode_plus(e.text_a, add_special_tokens=True, max_length=128, return_token_type_ids=True, return_attention_mask=True, pad_to_max_length=True, truncation=True)
        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"], input_dict["token_type_ids"], input_dict['attention_mask'])
        features.append(InputFeatures(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label))

    def gen():
        for f in features:
            yield ({"input_ids": f.input_ids, "attention_mask": f.attention_mask, "token_type_ids": f.token_type_ids}, f.label)

    return tf.data.Dataset.from_generator(gen, ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64))

In [14]:
DATA_COLUMN = 'Text'
LABEL_COLUMN = 'Sentiment'
label_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2, 'Extremely Positive': 3, 'Extremely Negative': 4}

# Drop rows where the text column has NaN values
train_data = train_data.dropna(subset=[DATA_COLUMN])
train_data[DATA_COLUMN] = train_data[DATA_COLUMN].fillna("")

# Proceed with your existing code...
train_data[LABEL_COLUMN] = train_data[LABEL_COLUMN].map(label_map)
train_InputExamples = convert_data_to_examples(train_data, DATA_COLUMN, LABEL_COLUMN)
train_data_tf = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)

d:\ProgrammFiles\miniconda\envs\cv\lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [15]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=["accuracy"])

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


OperatorNotAllowedInGraphError: in user code:

    File "d:\ProgrammFiles\miniconda\envs\cv\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "d:\ProgrammFiles\miniconda\envs\cv\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\ProgrammFiles\miniconda\envs\cv\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "d:\ProgrammFiles\miniconda\envs\cv\lib\site-packages\transformers\modeling_tf_utils.py", line 1637, in train_step
        y_pred = self(x, training=True)
    File "d:\ProgrammFiles\miniconda\envs\cv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    OperatorNotAllowedInGraphError: Exception encountered when calling layer 'tf_bert_for_seq

In [ ]:
model.fit(train_data_tf.shuffle(1000).batch(16), epochs=3, batch_size=16)

In [ ]:
validation_InputExamples = convert_data_to_examples_without_labels(test_data, DATA_COLUMN)
test_data_tf = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)

predictions = model.predict(test_data_tf.batch(16))
predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

label_reverse_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2, 'Extremely Positive': 3, 'Extremely Negative': 4}
predicted_labels_str = [label_reverse_map[label] for label in predicted_labels]